In [1]:
import os
import pandas as pd
import sys
    
from dbfread import DBF


In [2]:
_reImportBQ = False

if _reImportBQ: 

    sys.path.insert(0, '..\..\Resources\Python-Functions')
    import BigQuery

    # import transit route outputs
    bqclient = BigQuery.getBigQueryClient_TDMScenarios()

    dfScenarios = bqclient.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.scenarios").to_dataframe()
    #dfScenarios
    dfScenarios.to_csv("_dfScenarios.csv", index=False)

    dfODDetail = bqclient.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_od_detail").to_dataframe()
    dfODDetail.to_csv("_dfODDetail.csv", index=False)
    #display(dfODDetail)
    dfPADetail = bqclient.query("SELECT * FROM tdm-scenarios.tdm_scenarios_output.route_pa_detail").to_dataframe()
    dfPADetail.to_csv("_dfPADetail.csv", index=False)
    #display(dfPADetail)
else:
    dfScenarios = pd.read_csv("_dfScenarios.csv")
    dfODDetail = pd.read_csv("_dfODDetail.csv")
    dfPADetail = pd.read_csv("_dfPADetail.csv")

In [3]:
# read in manully created file
dfRouteSummaryNames_Revised = pd.read_csv(r"E:\GitHub\TDM-Scenario-Comparison-Toolbox\0-ModelModifications\manual\routesummarynames_revised.csv")
#dfRouteSummaryNames_Revised

In [4]:
paDropColumns = ['NAME',
                 'LONGNAME'     ,
                 'numRecords'   ,
                 'minYear'      ,
                 'maxYear'      ,
                 'minMode'      ,
                 'NAMEwithZeros',
                 'ROUTEID'      ,
                 'NAMEID'       ,
                 'OPERATOR'     ,
                 'PK_HEADWAY'   ,
                 'OK_HEADWAY'   ,
                 'PK_TIME'      ,
                 'OK_TIME'      ,
                 'DIST'         ,
                 'PK_D_BRDA'    ,
                 'PK_W_BRDA'    ,
                 'PK_BRDA'      ,
                 'PK_XITB'      ,
                 'OK_D_BRDA'    ,
                 'OK_W_BRDA'    ,
                 'OK_BRDA'      ,
                 'OK_XITB'      ,
                 'DY_D4_BRDA'   ,
                 'DY_W4_BRDA'   ,
                 'DY_4_XITB'    ,
                 'DY_D5_BRDA'   ,
                 'DY_W5_BRDA'   ,
                 'DY_5_XITB'    ,
                 'DY_D6_BRDA'   ,
                 'DY_W6_BRDA'   ,
                 'DY_6_XITB'    ,
                 'DY_D7_BRDA'   ,
                 'DY_W7_BRDA'   ,
                 'DY_7_XITB'    ,
                 'DY_D8_BRDA'   ,
                 'DY_W8_BRDA'   ,
                 'DY_8_XITB'    ,
                 'DY_D9_BRDA'   ,
                 'DY_W9_BRDA'   ,
                 'DY_9_XITB'    ,
                 'DY_D_BRDA'    ,
                 'DY_W_BRDA'    ,
                 'DY_BRDA'      ,
                 'DY_XITB'      ]


In [5]:
dfODDetailWithRevNames = pd.DataFrame.merge(dfODDetail,dfRouteSummaryNames_Revised,on='NAME')
dfODDetailWithRevNames = dfODDetailWithRevNames.drop(columns=('NAME'))
dfODDetailWithRevNames = dfODDetailWithRevNames.rename(columns={'NAME_New':'NAME'})
#display(dfODDetailWithRevNames)

# change negative to column
dfPADetail.loc[(dfPADetail['NAME'].str.strip().str[-1]!='-'), 'Direction'] = 'P'
dfPADetail.loc[(dfPADetail['NAME'].str.strip().str[-1]=='-'), 'Direction'] = 'N'
dfPADetail.loc[(dfPADetail['NAME'].str.strip().str[-1]=='-'), 'NAME'] = dfPADetail['NAME'].str[:-1]

dfPADetailWithRevNames = pd.DataFrame.merge(dfPADetail,dfRouteSummaryNames_Revised,on='NAME')
dfPADetailWithRevNames = dfPADetailWithRevNames.drop(columns=(paDropColumns))
dfPADetailWithRevNames = dfPADetailWithRevNames.rename(columns={'NAME_New':'NAME'})
#display(dfPADetailWithRevNames)

In [6]:
dfTotVol = pd.DataFrame()

# loop through scenarios and calc Link A and B -- USE Transit-Map TransitRidership.ipynb methodology
for index, row in dfScenarios.iterrows():

    dfTemp = dfODDetailWithRevNames[dfODDetailWithRevNames['scenarioID']==row['scenarioID']]
    #display(dfTemp)

    dfLinkLookup = dfTemp[['scenarioID','MODE','NAMEID','NAME','LINKSEQ1','A']].copy()
    dfLinkLookup['LINKSEQ_OF_B'] = dfLinkLookup['LINKSEQ1'] + 1

    dfLinkLookupMax = dfLinkLookup.groupby(['scenarioID','NAMEID','NAME'],as_index=False).agg(MAXLINKSEQ1=('LINKSEQ1','max'))
    dfLinkLookupMax['MAXLINKSEQp1'] = dfLinkLookupMax['MAXLINKSEQ1'] + 1
    dfLinkLookupWithMaxPlus1 = pd.DataFrame.merge(dfLinkLookup, dfLinkLookupMax, on=('scenarioID','NAMEID','NAME'), how='left')
    dfLinkLookupWithMaxPlus1.loc[(dfLinkLookupWithMaxPlus1['LINKSEQ1']==0), 'LINKSEQ1'] = dfLinkLookupWithMaxPlus1['MAXLINKSEQp1']
    dfLinkLookupWithMaxPlus1.loc[(dfLinkLookupWithMaxPlus1['LINKSEQ_OF_B']==1), 'LINKSEQ_OF_B'] = 0

    dfLinkLookupWithMaxPlus1 = dfLinkLookupWithMaxPlus1.drop(columns=['MAXLINKSEQ1','MAXLINKSEQp1'])

    dfJoinToGetB = pd.DataFrame.merge(dfLinkLookupWithMaxPlus1, dfLinkLookupWithMaxPlus1, left_on=('MODE','NAMEID','NAME','LINKSEQ_OF_B'), right_on=('MODE','NAMEID','NAME','LINKSEQ1'))
    dfJoinToGetB = dfJoinToGetB[['MODE','NAMEID','NAME','LINKSEQ1_x','A_x','A_y']]
    dfJoinToGetB.columns = ('MODE','NAMEID','NAME','LINKSEQ1','A','B')

    dfTempPrepped = pd.DataFrame.merge(dfJoinToGetB,dfTemp[['MODE','NAMEID','NAME','LINKSEQ1','A','DIST','D_TOT_VOL','scenarioID']], on=('MODE','NAMEID','NAME','LINKSEQ1','A'))

    #display(dfLinkLookup)
    #display(dfLinkLookupMax)
    #display(dfLinkLookupWithMaxPlus1)
    #display(dfJoinToGetB)
    #display(dfTempPrepped)

    dfTotVol = dfTotVol.append(dfTempPrepped)

    #display(dfLinkLookup.sort_values(by='LINK', ascending=False).drop_duplicates(subset='NAME'))

display(dfTotVol)
display(dfTotVol['D_TOT_VOL'].sum())

,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,scenarioID
0,4,4.0,DA605F,1,20131,3551,0.11,114.99,1
1,4,4.0,DA605F,6,8979,3411,0.09,119.31,1
2,4,4.0,DA605F,10,3415,9693,0.18,91.33,1
3,4,4.0,DA605F,11,9693,3417,0.17,78.85,1
4,4,4.0,DA605F,12,3417,3570,0.14,70.61,1
...,...,...,...,...,...,...,...,...,...
8223,4,16.0,UT809F,94,68565,68576,0.12,13.81,32
8224,4,16.0,UT809F,95,68576,68608,0.10,13.81,32
8225,4,16.0,UT809F,96,68608,68651,0.10,13.81,32
8226,4,16.0,UT809F,97,68651,68670,0.09,13.81,32


176933078.59000003

In [7]:
# Add SegIDs
#dfNetWithSegIDs = pd.DataFrame(DBF(r"A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-12-19\1_Inputs\3_Highway\GIS\MasterNet_v9 - 2023-01-26 - Link.dbf", load=True))
dfNetWithSegIDs = pd.DataFrame(DBF(r"A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-12-19\Scenarios\Needs_2050\0_InputProcessing\v9_Needs_SE50_Net50_Link.DBF", load=True))
dfNetWithSegIDs = dfNetWithSegIDs[['A','B','SEGID','IB_OB']]
#dfNetWithSegIDs = dfNetWithSegIDs[dfNetWithSegIDs['SEGID']!='']
dfNetWithSegIDs

,A,B,SEGID,IB_OB
0,1,29685,,IB
1,2,29685,,OB
2,3,29645,,IB
3,4,29671,,OB
4,5,29680,,OB
...,...,...,...,...
50532,95064,95025,,IB
50533,95064,95026,,OB
50534,95064,95056,,OB
50535,95065,3621,,IB


In [8]:
# join od detail with link seg data. only preserve rows with data in both tables since some links don't have segids etc
dfTotVolWithSegIDs = pd.DataFrame.merge(dfTotVol,dfNetWithSegIDs,on=('A','B'),how='inner')
dfTotVolWithSegIDs['DISTxVOL'] = dfTotVolWithSegIDs['DIST'] * dfTotVolWithSegIDs['D_TOT_VOL']
dfTotVolWithSegIDs

,MODE,NAMEID,NAME,LINKSEQ1,A,B,DIST,D_TOT_VOL,scenarioID,SEGID,IB_OB,DISTxVOL
0,4,13.0,UT834,37,23870,23869,0.24,128.85,1,2218_000.5,IB,30.9240
1,4,30.0,MAG_017,58,23870,23869,0.24,101.89,2,2218_000.5,IB,24.4536
2,4,31.0,MAG_018,35,23870,23869,0.24,130.05,2,2218_000.5,IB,31.2120
3,4,30.0,MAG_017,56,23870,23869,0.24,114.70,3,2218_000.5,IB,27.5280
4,4,31.0,MAG_018,35,23870,23869,0.24,144.72,3,2218_000.5,IB,34.7328
...,...,...,...,...,...,...,...,...,...,...,...,...
217486,4,7.0,WFRC_037,135,62338,62341,0.51,35.17,25,0145_003.8,IB,17.9367
217487,4,1.0,WFRC_037,136,62341,71513,0.46,33.60,24,0145_004.4,IB,15.4560
217488,4,7.0,WFRC_037,136,62341,71513,0.46,35.17,25,0145_004.4,IB,16.1782
217489,4,1.0,WFRC_037,137,71513,62346,0.15,33.60,24,0145_004.8,IB,5.0400


In [9]:
dfTotVolStatsBySegID = dfTotVolWithSegIDs.groupby(['scenarioID','MODE','NAME','SEGID'],as_index=False).agg(DISTxVOL=('DISTxVOL','sum'),DIST=('DIST','sum'))
dfTotVolStatsBySegID['SEGVOL'] = round(dfTotVolStatsBySegID['DISTxVOL'] / dfTotVolStatsBySegID['DIST'],0).astype(int)
dfTotVolStatsBySegID.drop(columns=['DISTxVOL','DIST'],inplace=True)
dfTotVolStatsBySegID

,scenarioID,MODE,NAME,SEGID,SEGVOL
0,1,4,UT806,WFRC_8222,58
1,1,4,UT834,2218_000.5,129
2,1,4,UT862,2080_005.7,201
3,1,7,701_Blue,2030_002.7,5434
4,1,7,701_Blue,2030_004.6,7362
...,...,...,...,...,...
64427,32,9,WE603,3400_000.1,1151
64428,32,9,WE603,3400_000.3,1171
64429,32,9,WE603,3400_000.4,1423
64430,32,9,WE603,3400_000.6,1423


In [10]:
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID.groupby(['scenarioID','SEGID','MODE'], as_index=False).agg(SEGVOLBYMODE=('SEGVOL','sum'))
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.pivot_table(index=('scenarioID','SEGID'), columns='MODE')
dfTotVolStatsBySegID_byMode.fillna(0,inplace=True)
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.astype(int)
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.reset_index()
dfTotVolStatsBySegID_byMode = dfTotVolStatsBySegID_byMode.droplevel(0, axis=1) 
dfTotVolStatsBySegID_byMode.columns = ['scenarioID','SEGID','M4','M5','M6','M7','M8','M9']
dfTotVolStatsBySegID_byMode['MT'] =  dfTotVolStatsBySegID_byMode['M4'] + \
                                     dfTotVolStatsBySegID_byMode['M5'] + \
                                     dfTotVolStatsBySegID_byMode['M6'] + \
                                     dfTotVolStatsBySegID_byMode['M7'] + \
                                     dfTotVolStatsBySegID_byMode['M8'] + \
                                     dfTotVolStatsBySegID_byMode['M9']
dfTotVolStatsBySegID_byMode

,scenarioID,SEGID,M4,M5,M6,M7,M8,M9,MT
0,1,0048_004.1,0,0,0,0,0,3124,3124
1,1,2030_002.7,0,0,0,12550,0,0,12550
2,1,2030_004.6,0,0,0,14565,0,0,14565
3,1,2030_005.1,0,0,0,4166,0,0,4166
4,1,2031_002.2,0,0,0,3134,0,0,3134
...,...,...,...,...,...,...,...,...,...
39060,32,WFRC_8227,562,0,0,0,0,0,562
39061,32,WFRC_8256,34,0,0,0,0,0,34
39062,32,WFRC_8258,100,0,0,0,0,0,100
39063,32,WFRC_8260,115,0,0,0,0,0,115


In [11]:
# read in json ids
dfWABScenarios = pd.read_json(r"E:\WebAppBuilderForArcGIS_v2.20\server\apps\8\widgets\tttScenarioManager\data\scenarios.json")
#display(dfWABScenarios)

# match with BigQuery Scenarios

dfBigQueryToWAB = pd.DataFrame([
    [1 , 'v8.3.2 BY 2019'       ],
    [2 , 'v8.3.2 RTP 2030'      ],
    [3 , 'v8.3.2 RTP 2040'      ],
    [4 , 'v8.3.2 RTP 2050'      ],
    [5 , 'v8.3.2 TIP 2024'      ],
    [18, 'v9 Beta BY 2019'      ],
    [19, 'v9 Beta BY 2023'      ],
    [20, 'v9 Beta RTP+EF 2032'  ],
    [21, 'v9 Beta RTP+EF 2042'  ],
    [22, 'v9 Beta RTP+EF 2050'  ],
    [23, 'v9 Beta Needs 2032'   ],
    [24, 'v9 Beta Needs 2042'   ],
    [25, 'v9 Beta Needs 2050'   ],
    [26, 'v9 Beta No Build 2032'],
    [27, 'v9 Beta No Build 2042'],
    [28, 'v9 Beta No Build 2050'],
    [29, 'v9 Beta RTP 2032'     ],
    [30, 'v9 Beta RTP 2042'     ],
    [31, 'v9 Beta RTP 2050'     ],
    [32, 'v9 Beta TIP 2028'     ]
],columns=('scenarioID', 'label'))
#display(dfBigQueryToWAB)

dfJoin = pd.DataFrame.merge(dfBigQueryToWAB,dfWABScenarios,on='label',how='left')
dfJoin = dfJoin[['scenarioID','value']]
dfJoin

,scenarioID,value
0,1,id_12066090786226718a9b4755.54851854
1,2,id_602780967622672681e6327.84737895
2,3,id_4927651456226727eb56079.08455813
3,4,id_16231621306226729673c147.07236076
4,5,id_3989814362267228a9f4b5.00876734
5,18,id_37001244863cadd40e1b8f7.52642706
6,19,id_72534583163cb54226ea202.98630416
7,20,id_73730688563cb5d2222cdc4.25550859
8,21,id_71963308763cb5d35348012.40299726
9,22,id_23888985563cb5d49295ba5.99558740


In [12]:
dfJoin[dfJoin['scenarioID']==2].value

1    id_602780967622672681e6327.84737895
Name: value, dtype: object

In [13]:
export = False

_outputdir_mode = os.path.join(os.getcwd(),'TTTDataTransitDetailByMode')
_outputdir_route = os.path.join(os.getcwd(),'TTTDataTransitDetailByRoute')

# loop through scenarios and calc Link A and B -- USE Transit-Map TransitRidership.ipynb methodology
for index, row in dfScenarios.iterrows():

    try:
        #print( dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist())
        _id = dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist()[0]
        #print(_id)
        _file = _id + '.json'
        #print(_file)
        _outputfile_mode = os.path.join(_outputdir_mode,_file)
        _outputfile_route = os.path.join(_outputdir_route,_file)

        dfModeOut = dfTotVolStatsBySegID_byMode[dfTotVolStatsBySegID_byMode['scenarioID']==row['scenarioID']]
        dfModeOut = dfModeOut.drop(columns=('scenarioID'))
        dfModeOut = dfModeOut.set_index('SEGID')
        if export: dfModeOut.to_json(_outputfile_mode,orient="table") #,orient='records'

        dfRouteOut = dfTotVolStatsBySegID[dfTotVolStatsBySegID['scenarioID']==row['scenarioID']]
        dfRouteOut = dfRouteOut.drop(columns=('scenarioID'))
        dfRouteOut = dfRouteOut.set_index('SEGID')
        if export: dfRouteOut.to_json(_outputfile_route,orient="table") #,orient='records'
        #display(dfTemp)
    except:
        print('scenarioID ' + str(row['scenarioID']) + ' error')
        continue

scenarioID 6 error
scenarioID 7 error
scenarioID 8 error
scenarioID 9 error
scenarioID 10 error
scenarioID 11 error
scenarioID 12 error
scenarioID 13 error
scenarioID 14 error
scenarioID 15 error
scenarioID 16 error
scenarioID 17 error


In [14]:
dfTotVolStatsBySegID_byMode[dfTotVolStatsBySegID_byMode['SEGID']=='WFRC_8227']

,scenarioID,SEGID,M4,M5,M6,M7,M8,M9,MT
1545,6,WFRC_8227,834,0,0,0,0,0,834
2835,7,WFRC_8227,926,0,0,0,0,0,926
4198,8,WFRC_8227,633,0,0,0,0,0,633
5607,9,WFRC_8227,611,0,0,0,0,0,611
7094,10,WFRC_8227,678,0,0,0,0,0,678
8556,11,WFRC_8227,1098,0,0,0,0,0,1098
10017,12,WFRC_8227,1318,0,0,0,0,0,1318
11478,13,WFRC_8227,1412,0,0,0,0,0,1412
12977,14,WFRC_8227,640,0,0,0,0,0,640
14528,15,WFRC_8227,601,0,0,0,0,0,601


In [15]:
dfPADetail_melt = dfPADetailWithRevNames
dfPADetail_melt = dfPADetail_melt.melt(id_vars=['scenarioID','NAME','Direction','MODE','LINKSEQ','A','B','STOPA','STOPB'], var_name='TransitVariable', value_name='TransitValue')
dfPADetail_melt
#dfPADetail_melt.columns

,scenarioID,NAME,Direction,MODE,LINKSEQ,A,B,STOPA,STOPB,TransitVariable,TransitValue
0,1,DA461,P,4,16,3423,3579,1,1,PK_D4_BRDA,0.0
1,1,DA461,P,4,17,3579,5516,1,1,PK_D4_BRDA,0.0
2,1,DA461,P,4,49,6784,3580,1,1,PK_D4_BRDA,0.0
3,1,DA461,P,4,64,3463,3486,1,1,PK_D4_BRDA,0.0
4,1,DA461,P,4,159,3486,3463,1,1,PK_D4_BRDA,0.0
...,...,...,...,...,...,...,...,...,...,...,...
14059867,31,MAG_002,N,9,14,63466,67894,0,0,OK_9_XITB,0.0
14059868,31,MAG_002,N,9,15,67894,67873,0,0,OK_9_XITB,0.0
14059869,31,MAG_002,N,9,16,67873,67863,0,0,OK_9_XITB,0.0
14059870,31,MAG_002,N,9,17,67863,67858,0,0,OK_9_XITB,0.0


In [16]:
dfVarToFields = pd.DataFrame([
 ['PK_D4_BRDA'   , 'PK', 'D', 4, 'BRDA'],
 ['PK_W4_BRDA'   , 'PK', 'W', 4, 'BRDA'],
 ['PK_4_XITB'    , 'PK', 'X', 4, 'XITB'],
 ['PK_D5_BRDA'   , 'PK', 'D', 5, 'BRDA'],
 ['PK_W5_BRDA'   , 'PK', 'W', 5, 'BRDA'],
 ['PK_5_XITB'    , 'PK', 'X', 5, 'XITB'],
 ['PK_D6_BRDA'   , 'PK', 'D', 6, 'BRDA'],
 ['PK_W6_BRDA'   , 'PK', 'W', 6, 'BRDA'],
 ['PK_6_XITB'    , 'PK', 'X', 6, 'XITB'],
 ['PK_D7_BRDA'   , 'PK', 'D', 7, 'BRDA'],
 ['PK_W7_BRDA'   , 'PK', 'W', 7, 'BRDA'],
 ['PK_7_XITB'    , 'PK', 'X', 7, 'XITB'],
 ['PK_D8_BRDA'   , 'PK', 'D', 8, 'BRDA'],
 ['PK_W8_BRDA'   , 'PK', 'W', 8, 'BRDA'],
 ['PK_8_XITB'    , 'PK', 'X', 8, 'XITB'],
 ['PK_D9_BRDA'   , 'PK', 'D', 9, 'BRDA'],
 ['PK_W9_BRDA'   , 'PK', 'W', 9, 'BRDA'],
 ['PK_9_XITB'    , 'PK', 'X', 9, 'XITB'],
 ['OK_D4_BRDA'   , 'OK', 'D', 4, 'BRDA'],
 ['OK_W4_BRDA'   , 'OK', 'W', 4, 'BRDA'],
 ['OK_4_XITB'    , 'OK', 'X', 4, 'XITB'],
 ['OK_D5_BRDA'   , 'OK', 'D', 5, 'BRDA'],
 ['OK_W5_BRDA'   , 'OK', 'W', 5, 'BRDA'],
 ['OK_5_XITB'    , 'OK', 'X', 5, 'XITB'],
 ['OK_D6_BRDA'   , 'OK', 'D', 6, 'BRDA'],
 ['OK_W6_BRDA'   , 'OK', 'W', 6, 'BRDA'],
 ['OK_6_XITB'    , 'OK', 'X', 6, 'XITB'],
 ['OK_D7_BRDA'   , 'OK', 'D', 7, 'BRDA'],
 ['OK_W7_BRDA'   , 'OK', 'W', 7, 'BRDA'],
 ['OK_7_XITB'    , 'OK', 'X', 7, 'XITB'],
 ['OK_D8_BRDA'   , 'OK', 'D', 8, 'BRDA'],
 ['OK_W8_BRDA'   , 'OK', 'W', 8, 'BRDA'],
 ['OK_8_XITB'    , 'OK', 'X', 8, 'XITB'],
 ['OK_D9_BRDA'   , 'OK', 'D', 9, 'BRDA'],
 ['OK_W9_BRDA'   , 'OK', 'W', 9, 'BRDA'],
 ['OK_9_XITB'    , 'OK', 'X', 9, 'XITB']
], columns=('TransitVariable','TimeOfDay','AccessMode','HeirarchyMode','BrdAXitB'))
dfVarToFields

,TransitVariable,TimeOfDay,AccessMode,HeirarchyMode,BrdAXitB
0,PK_D4_BRDA,PK,D,4,BRDA
1,PK_W4_BRDA,PK,W,4,BRDA
2,PK_4_XITB,PK,X,4,XITB
3,PK_D5_BRDA,PK,D,5,BRDA
4,PK_W5_BRDA,PK,W,5,BRDA
5,PK_5_XITB,PK,X,5,XITB
6,PK_D6_BRDA,PK,D,6,BRDA
7,PK_W6_BRDA,PK,W,6,BRDA
8,PK_6_XITB,PK,X,6,XITB
9,PK_D7_BRDA,PK,D,7,BRDA


# Riders by Link

In [17]:
dfPALong = pd.DataFrame.merge(dfPADetail_melt, dfVarToFields, on='TransitVariable')
dfPALong.drop(columns=('TransitVariable'),inplace=True)
dfPALong_pivot = dfPALong.pivot_table(index=('scenarioID','NAME','Direction','MODE','LINKSEQ','A','B','STOPA','STOPB','TimeOfDay','HeirarchyMode'), columns=('AccessMode','BrdAXitB'))
dfPALong_pivot.columns = dfPALong_pivot.columns.to_flat_index()
dfPALong_pivot = dfPALong_pivot.reset_index()

# rename some columns
dfPALong_pivot.columns = ('scenarioID','NAME','Direction','MODE','LINKSEQ','A','B','STOPA','STOPB','TimeOfDay','HeirarchyMode','D_BRDA','W_BRDA','X_XITB')

dfPALong_pivot_withIBOB = pd.DataFrame.merge(dfPALong_pivot,dfNetWithSegIDs,on=('A','B'),how='left')
dfPALong_pivot_withIBOB = dfPALong_pivot_withIBOB.fillna(0)

In [18]:
# CALCULATE RIDERS BY LINK

# column denifinitions
_keyColumns = ['scenarioID','TimeOfDay','MODE','NAME','Direction','HeirarchyMode']
_sortColumns = _keyColumns + ['LINKSEQ']

# sort by for transit direcition
dfPAPivot = dfPALong_pivot_withIBOB.sort_values(by=_sortColumns)

# shift exits to next link in order to facilitate rider calcs
dfPAPivot['X_XITB_Shift'] = dfPAPivot['X_XITB'].shift(1)

# set first link seq to zero because shift would have moved from previous route/time/direction/etc.
dfPAPivot.loc[dfPAPivot['LINKSEQ']==1,'X_XITB_Shift'] = 0

# calculate cummulative sums by key columns
dfPAPivot['D_BRDA_cumsum'      ] = dfPAPivot.groupby(_keyColumns)['D_BRDA'      ].transform(pd.Series.cumsum)
dfPAPivot['W_BRDA_cumsum'      ] = dfPAPivot.groupby(_keyColumns)['W_BRDA'      ].transform(pd.Series.cumsum)
dfPAPivot['X_XITB_Shift_cumsum'] = dfPAPivot.groupby(_keyColumns)['X_XITB_Shift'].transform(pd.Series.cumsum)

# riders is total of cumulative drive and walk boardings minus the shift in exits
dfPAPivot['Riders'] = dfPAPivot['D_BRDA_cumsum'] + dfPAPivot['W_BRDA_cumsum'] - dfPAPivot['X_XITB_Shift_cumsum']

In [19]:
dfPARiders = dfPAPivot.drop(columns=(['D_BRDA_cumsum','W_BRDA_cumsum','X_XITB_Shift_cumsum']))
dfPARiders['LinkID'] = dfPARiders['A'].astype(str) + '_' + dfPARiders['B'].astype(str)
dfPARiders = dfPARiders[['scenarioID','TimeOfDay','MODE','NAME','HeirarchyMode','LinkID','IB_OB','SEGID','Riders']]
dfPARiders['IB_OB'] = dfPARiders['IB_OB'].fillna('XX')
dfPARiders['SEGID'] = dfPARiders['SEGID'].fillna('')
dfPARiders.columns = ['s','tod','md','nm','hmd','lid','io','segid','r']
dfPARiders

,s,tod,md,nm,hmd,lid,io,segid,r
4584,1,OK,4,BE630,4,20142_6885,0,0,13.28
4596,1,OK,4,BE630,4,6885_6923,0,0,13.28
4608,1,OK,4,BE630,4,6923_6888,0,0,13.29
4620,1,OK,4,BE630,4,6888_9357,0,0,13.44
4632,1,OK,4,BE630,4,9357_6889,0,0,13.44
...,...,...,...,...,...,...,...,...,...
4672223,32,PK,9,WE603,9,28727_28678,OB,0203_002.3,35.79
4672235,32,PK,9,WE603,9,28678_28651,OB,0203_001.8,34.85
4672247,32,PK,9,WE603,9,28651_28631,OB,0203_001.8,9.83
4672259,32,PK,9,WE603,9,28631_28619,OB,0203_001.6,9.54


# Boardings/Alightings by Node

In [20]:
# calculate boardings
dfPABoardings  = dfPAPivot.groupby(_keyColumns + ['A','IB_OB','SEGID'],as_index=False).agg(D_BRD=('D_BRDA','sum'),W_BRD=('W_BRDA','sum'))
dfPABoardings = dfPABoardings[(dfPABoardings['D_BRD']>0) | (dfPABoardings['W_BRD']>0)]
#display(dfPABoardings)

# calculate alightings (exits)
dfPAAlightings = dfPAPivot.groupby(_keyColumns + ['B','IB_OB','SEGID'],as_index=False).agg(X_XIT=('X_XITB','sum'))
dfPAAlightings = dfPAAlightings[(dfPAAlightings['X_XIT']>0)]
#display(dfPAAlightings)

# combine
dfPABoardAlight = pd.DataFrame.merge(dfPABoardings, dfPAAlightings, left_on=(_keyColumns + ['A','IB_OB','SEGID']), right_on=(_keyColumns + ['B','IB_OB','SEGID']), how='outer')
dfPABoardAlight = dfPABoardAlight.fillna(0)
dfPABoardAlight['STOPNODE'] = dfPABoardAlight[['A', 'B']].max(axis=1).astype(int)
dfPABoardAlight = dfPABoardAlight[['scenarioID','TimeOfDay','MODE','NAME','IB_OB','SEGID','HeirarchyMode','STOPNODE','D_BRD','W_BRD','X_XIT']]
dfPABoardAlight.columns = ['s','tod','md','nm','io','segid','hmd','nid','db','wb','x']
dfPABoardAlight

,s,tod,md,nm,io,segid,hmd,nid,db,wb,x
0,1,OK,4,BE630,0,0,4,6548,0.0,0.45,0.11
1,1,OK,4,BE630,0,0,4,6888,0.0,0.15,0.00
2,1,OK,4,BE630,0,0,4,6923,0.0,0.01,0.00
3,1,OK,4,BE630,0,0,4,7055,0.0,1.44,5.44
4,1,OK,4,BE630,0,0,4,7095,0.0,0.24,0.21
...,...,...,...,...,...,...,...,...,...,...,...
800090,32,PK,9,WE603,OB,3354_002.6,9,28820,0.0,0.00,0.37
800091,32,PK,9,WE603,IB,3400_000.3,9,28823,0.0,0.00,3.96
800092,32,PK,9,WE603,OB,3400_000.6,9,28899,0.0,0.00,2.38
800093,32,PK,9,WE603,IB,3404_001.5,9,28935,0.0,0.00,0.91


# Export jsons

In [32]:
export = True

_outputdir_paLink = os.path.join(os.getcwd(),'_TTTDataTransitPALink')
_outputdir_paNode = os.path.join(os.getcwd(),'_TTTDataTransitPANode')
_outputdir_routeModeName = os.path.join(os.getcwd(),'_TTTRouteNameMode')

# loop through scenarios and calc Link A and B -- USE Transit-Map TransitRidership.ipynb methodology
for index, row in dfScenarios.iterrows():

    try:
        #print( dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist())
        if (len(dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist())>0):
            _id = dfJoin[dfJoin['scenarioID']==row['scenarioID']]['value'].tolist()[0]
            #print(_id)
            _file = _id + '.json'
            print(_file)
            _outputfile_paLink = os.path.join(_outputdir_paLink,_file)
            _outputfile_paNode = os.path.join(_outputdir_paNode,_file)
            _outputfile_routeModeName = os.path.join(_outputdir_routeModeName,_file)

            dfLinkOut = dfPARiders[dfPARiders['s']==row['scenarioID']]
            dfLinkOut = dfLinkOut.drop(columns=('s'))
            dfLinkOut = dfLinkOut.set_index('lid')
            if export: dfLinkOut.to_json(_outputfile_paLink,orient="table") #,orient='records'

            dfNodeOut = dfPABoardAlight[dfPABoardAlight['s']==row['scenarioID']]
            dfNodeOut = dfNodeOut.drop(columns=('s'))
            dfNodeOut = dfNodeOut.set_index('nid')
            if export: dfNodeOut.to_json(_outputfile_paNode,orient="table") #,orient='records'

            # Output routename list per scenario
            dfRouteOut = dfLinkOut[['nm','md']].drop_duplicates()
            dfRouteOut = pd.DataFrame.merge(dfRouteOut,dfRouteSummaryNames_Revised[['NAME_New','LONGNAME']],left_on='nm',right_on='NAME_New',how='left')
            dfRouteOut = dfRouteOut.drop(columns='NAME_New')
            dfRouteOut = dfRouteOut.rename(columns={'nm':'value','md':'mode','LONGNAME':'label'})
            dfRouteOut = dfRouteOut.set_index('value')
            if export: dfRouteOut.to_json(_outputfile_routeModeName,orient="table") #,orient='records'
            
            #display(dfTemp)
    except:
        print('scenarioID ' + str(row['scenarioID']) + ' error')
        continue

id_12066090786226718a9b4755.54851854.json
id_602780967622672681e6327.84737895.json
id_4927651456226727eb56079.08455813.json
id_16231621306226729673c147.07236076.json
id_3989814362267228a9f4b5.00876734.json
id_37001244863cadd40e1b8f7.52642706.json
id_72534583163cb54226ea202.98630416.json
id_73730688563cb5d2222cdc4.25550859.json
id_71963308763cb5d35348012.40299726.json
id_23888985563cb5d49295ba5.99558740.json
id_164525944263cb5a65b998b7.54218474.json
id_22075304363cb5a8a520834.45828991.json
id_63298618163cb5aa9ba1549.90065432.json
id_55480621263cb5b14c32686.99056477.json
id_161746222663cb5b26a7d924.02056464.json
id_199631216463cb5b3a4cfcb1.37627729.json
id_144890492163cb543f110d71.15198462.json
id_161237566463cb5461c885c0.97170851.json
id_98449779163cadd267afbd0.97795935.json
id_133633625463cb540774f8a1.20476248.json


In [29]:
dfRouteOut

,value,md,label
0,BE630,4,630 - Brigham City-Ogden Comm
1,DA460,4,460 - Woods Cross
2,DA461,4,461 - Bountiful via State Capitol
3,DA462,4,462 - North Salt Lake
4,DA605F,4,F605 - Bountiful - Centerville Flex
...,...,...,...
109,704_Green,7,704 - TRAX Green Line
110,720_SLine,7,720 - S-Line
111,750_FR,8,750 - FrontRunner
112,750_FR,8,750 - FrontRunner


In [22]:
dfPARiders[['s','md','nm']].drop_duplicates()

,s,md,nm
4584,1,4,BE630
6744,1,4,DA460
8304,1,4,DA461
10968,1,4,DA462
12432,1,4,DA605F
...,...,...,...
4549836,32,7,720_SLine
4550028,32,8,750_FR
4652652,32,9,UT830X
4654764,32,9,UT830X.1


# Export Link and Node Shapefiles

In [23]:

# Import the required ArcGIS API for Python modules
import pandas as pd

import arcgis
gis = arcgis.GIS()

sdfLink = pd.DataFrame.spatial.from_featureclass(r"A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-12-19\1_Inputs\3_Highway\GIS\MasterNet_v9 - 2023-02-02 - Link.shp")
sdfNode = pd.DataFrame.spatial.from_featureclass(r"A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-12-19\1_Inputs\3_Highway\GIS\MasterNet_v9 - 2023-02-02 - Node.shp")

In [24]:
export = False

if export:

    # scrape out unneeded data
    sdfLink = sdfLink[['LINKID','SHAPE']]
    sdfNode = sdfNode[['N','SHAPE']]

    # used links export
    sdfLinksForTransit = pd.DataFrame.merge(sdfLink,dfPARiders[['lid']].drop_duplicates(),left_on='LINKID',right_on='lid',how='inner')
    sdfLinksForTransit.drop(columns=('lid'),inplace=True)
    sdfLinksForTransit.spatial.to_featureclass(os.path.join('_tdm_network','transitlinks.shp'),sanitize_columns=False)

    # used nodes export
    sdfNodesForTransit = pd.DataFrame.merge(sdfNode,dfPABoardAlight[['nid']].drop_duplicates(),left_on='N',right_on='nid',how='inner')
    sdfNodesForTransit.drop(columns=('nid'),inplace=True)
    sdfNodesForTransit.spatial.to_featureclass(os.path.join('_tdm_network','transitnodes.shp'),sanitize_columns=False)

In [25]:
#dfToJson = dfPARiders[dfPARiders['scenarioID']==1].copy()
#dfToJson['ID'] = dfToJson['LinkID'] + "_" + dfToJson['TimeOfDay'] + "_" + dfToJson['MODE'].astype(str) + "_" + dfToJson['NAME'] + "_" + dfToJson['HeirarchyMode'].astype(str) + "_" + dfToJson['IB_OB']
#dfToJson = dfToJson[['ID','Riders']]
#dfToJson = dfToJson.drop_duplicates()
#print (dfToJson.set_index('ID').rename(columns={'Riders':'index1'}).to_json())
##{"index1":{"1":"a","2":"b","3":"c","4":"d"}}


